In [ ]:
# Run once

# !ssh-keygen -t rsa -b 4096 -f /content/drive/MyDrive/ML_Sem3_Project/.ssh/id_rsa -N "" -q
# !cat /content/drive/MyDrive/ML_Sem3_Project/.ssh/id_rsa.pub

In [2]:
# Run everytime

from google.colab import drive
drive.mount('/content/drive')

!mkdir -p ~/.ssh
!cp /content/drive/MyDrive/ML_Sem3_Project/.ssh/id_rsa ~/.ssh/
!cp /content/drive/MyDrive/ML_Sem3_Project/.ssh/id_rsa.pub ~/.ssh/
!chmod 600 ~/.ssh/id_rsa
!ssh-keyscan github.com >> ~/.ssh/known_hosts

import os

repo_dir = "/content/ML_Project_2025"

if not os.path.exists(repo_dir):
  !git clone git@github.com:astroartics/ML_Project_2025.git {repo_dir}
else:
  %cd {repo_dir}
  !git pull

%cd {repo_dir}

Mounted at /content/drive
# github.com:22 SSH-2.0-1e748d5
# github.com:22 SSH-2.0-1e748d5
# github.com:22 SSH-2.0-1e748d5
# github.com:22 SSH-2.0-1e748d5
# github.com:22 SSH-2.0-1e748d5
Cloning into '/content/ML_Project_2025'...
remote: Enumerating objects: 170, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 170 (delta 90), reused 121 (delta 41), pack-reused 0 (from 0)
Receiving objects: 100% (170/170), 1.57 MiB | 5.35 MiB/s, done.
Resolving deltas: 100% (90/90), done.
/content/ML_Project_2025


In [3]:

# Run everytime

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("\nUsing device:", device, "\n")

checkpoint_path = "/content/drive/MyDrive/ML_Sem3_Project/preprocessed_data.pt"
checkpoint = torch.load(
    checkpoint_path,
    map_location = device,  # ensures safe loading on CPU/GPU
    weights_only = False
)

# --------------------------------

# 4. Extract Variables
vocab = checkpoint["vocab"]
inv_vocab = checkpoint["inv_vocab"]

# Move tensors to device
X_padded = checkpoint["X_padded"].to(device)  # PyTorch does not run natively on TPU, hence using T4 GPU
y_tensor = checkpoint["y_tensor"].to(device)

# DataFrame cannot move to device; it stays on CPU
data = checkpoint["data"]

print(data["moves"].head())

# --------------------------------

# 5. Status
print("\nLoaded checkpoint successfully.\n")
print("X_padded shape:", X_padded.shape)
print("y_tensor shape:", y_tensor.shape)
print("Example vocab size:", len(vocab))


Using device: cuda 

0    e4 d5 Nf3 dxe4 Ne5 Nf6 d4 exd3 Bxd3 e6 Nc3 Bd6...
1    b4 e5 Bb2 d6 c3 Bf5 d3 Nf6 e4 Bg6 Be2 Be7 Nf3 ...
2    e4 d5 exd5 Qxd5 Nc3 Qa5 Nf3 Nf6 d4 Bg4 Bd2 Nc6...
3    e4 e5 Nf3 Nf6 Nc3 d6 Bc4 Be6 Qe2 Nbd7 d4 Bxc4 ...
4    d4 e6 c4 c6 Nc3 d5 cxd5 cxd5 e4 Nc6 e5 Qb6 Nge...
Name: moves, dtype: object

Loaded checkpoint successfully.

X_padded shape: torch.Size([658379, 292])
y_tensor shape: torch.Size([658379])
Example vocab size: 3672


In [8]:
# Push changes to GitHub

!git config --global user.email "joshisanjanana114@gmail.com"
!git config --global user.name "Sanjana"

!cp /content/drive/MyDrive/ML_Sem3_Project/BaselineLSTM.ipynb /content/ML_Project_2025/

!git add BaselineLSTM.ipynb
!git commit -m "Added Notebook"
!git push

[main a4be07e] Added Notebook
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite BaselineLSTM.ipynb (99%)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 2.51 KiB | 2.51 MiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:astroartics/ML_Project_2025.git
   5b531a4..a4be07e  main -> main


In [3]:
# Train/test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_padded,
    y_tensor,
    test_size = 0.2,  # 80-20 split
    random_state = 42,
    shuffle=True
)

X_train.shape, X_test.shape


(torch.Size([526703, 292]), torch.Size([131676, 292]))

In [5]:
# PyTorch Datasets and DataLoaders

# Why these are needed:
'''
1) Data cannot be fed to the model directly, because the number of integers is huge.
2) It must be given to the model in batches.
3) A DataLoader iterates over these batches.
4) DataLoader handles batching, shuffling, and feeding the model (if batch size = 64, then taking 64 sequences for X and 64 labels for y)
5) DataLoader also uses multiple workers to load batches faster.
6) Dataset allows Python to treat the data like a list (shuffling indices, slicing samples, fetching mini-batches automatically)
7) Dataset is a wrapper that tells PyTorch "How to get the ith sample."
8) Dataset: Defines how to access each sample
   DataLoader: Defines how to group examples into batches efficiently during training
'''

from torch.utils.data import Dataset, DataLoader

# X and y are not converted to a new format, they are just wrapped and become accessible through the Dataset interface
# For example, before Dataset, we manually index
# sample_X = X_train[100]
# sample_y = y_train[100]
# but, after Dataset, DataLoader can automatically fetch train_ds[100] -> returns (X_train[100], y_train[100])
class ChessDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    # Returns one sample to the DataLoader when requested: DataLoader needs data to be wrapped in a Dataset to
    # access it one by one randomly to form batches.
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# DataLoaders

batch_size = 64

train_dataset = ChessDataset(X_train, y_train)
test_dataset  = ChessDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size = batch_size)


In [1]:
import torch
import torch.nn as nn

# Defining the Baseline LSTM model

class BaselineLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256, num_layers=1, padding_idx=0):
        super().__init__()
        # Embedding: converts token IDs -> dense vectors
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim, padding_idx=padding_idx)
        # LSTM: processes sequence of embeddings
        self.lstm = nn.LSTM(input_size=embed_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
        # Final linear layer to produce logits for each vocabulary token
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        # x: LongTensor of shape (batch_size, seq_len)
        emb = self.embedding(x)                     # -> (batch_size, seq_len, embed_dim)
        outputs, (h_n, c_n) = self.lstm(emb)        # outputs -> (batch_size, seq_len, hidden_dim)
                                                   # h_n -> (num_layers, batch_size, hidden_dim)
        last_hidden = outputs[:, -1, :]             # take output at last time-step -> (batch_size, hidden_dim)
        logits = self.fc(last_hidden)               # -> (batch_size, vocab_size)
        return logits

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab_size = len(vocab)        # vocab from your preprocessing (includes PAD and UNK)
model = BaselineLSTM(vocab_size=vocab_size, embed_dim=128, hidden_dim=256, num_layers=1, padding_idx=vocab['<PAD>']).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=vocab['<PAD>'])   # ignore pad index if used in targets (usually not)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
num_epochs = 10
model.train()
for epoch in range(num_epochs):
    total_loss = 0.0
    for X_batch, y_batch in train_loader:
        # Move to device
        X_batch = X_batch.to(device)                   # (batch_size, seq_len), dtype=torch.long
        y_batch = y_batch.to(device)                   # (batch_size,), dtype=torch.long (class indices)

        optimizer.zero_grad()

        logits = model(X_batch)                        # (batch_size, vocab_size)
        loss = criterion(logits, y_batch)              # CrossEntropy expects (N, C) and targets (N,)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * X_batch.size(0)    # sum-of-losses for averaging later

    avg_loss = total_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}  Train loss: {avg_loss:.4f}")

# Predicting / decoding one example
model.eval()
with torch.no_grad():
    sample_X = X_test[0].unsqueeze(0).to(device)      # shape (1, seq_len)
    logits = model(sample_X)                          # (1, vocab_size)
    pred_id = logits.argmax(dim=-1).item()            # integer token id
    pred_move = inv_vocab[pred_id]                    # decode to SAN using your inv_vocab
    print("Predicted token id:", pred_id, "Predicted SAN:", pred_move)


NameError: name 'vocab' is not defined